Testowanie API

In [1]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# 1. Inicjalizacja Sparka ze sterownikiem do Postgresa
spark = SparkSession.builder \
    .appName("NBP_Test") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0") \
    .getOrCreate()

# 2. Pobranie danych z API (ostatnie 30 dni kursu Euro)
url = "http://api.nbp.pl/api/exchangerates/rates/a/eur/last/30/?format=json"
response = requests.get(url).json()

# 3. Przerobienie JSON na DataFrame Sparka
df_raw = spark.createDataFrame(response['rates'])

# 4. Prosta transformacja (zmiana typów i dodanie kodu waluty)
df_silver = df_raw.select(
    F.lit(response['code']).alias("currency"),
    F.col("effectiveDate").cast("date").alias("date"),
    F.col("mid").cast("float").alias("rate")
)

df_silver.show(5)


+--------+----------+------+
|currency|      date|  rate|
+--------+----------+------+
|     EUR|2025-11-18|4.2331|
|     EUR|2025-11-19|4.2424|
|     EUR|2025-11-20|4.2289|
|     EUR|2025-11-21|4.2489|
|     EUR|2025-11-24|4.2345|
+--------+----------+------+
only showing top 5 rows



In [2]:
# Zapisujemy do kontenera postgres_dw (port 5432, bo jesteśmy wewnątrz sieci Docker)
df_silver.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres_dw:5432/currency_db") \
    .option("dbtable", "test_nbp_rates") \
    .option("user", "admin") \
    .option("password", "password123") \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

print(" Sukces! Dane zostały zapisane w Postgresie.")

✅ Sukces! Dane zostały zapisane w Postgresie.


Pobranie obecnych walut dostępnych w API z tabel A i B

In [8]:
import requests
import json
from pathlib import Path

# 1. Definiujemy BEZWZGLĘDNĄ ścieżkę (taką, jaką widzi Jupyter)
# To jest ten sam folder, który masz na dysku C:\Projects\NBP_spark\data\bronze
bronze_path = Path("/home/jovyan/work/data/bronze")

# 2. Tworzymy folder, jeśli go nie ma (parents=True stworzy też folder 'data' jeśli trzeba)
bronze_path.mkdir(parents=True, exist_ok=True)

# 3. Pobieramy dane
print("Pobieram dane z NBP...")
raw_data_a = requests.get("http://api.nbp.pl/api/exchangerates/tables/a/?format=json").json()

# 4. Zapisujemy używając pełnej ścieżki
file_to_save = bronze_path / "raw_table_a.json"

with open(file_to_save, 'w', encoding='utf-8') as f:
    json.dump(raw_data_a, f, ensure_ascii=False, indent=4)

print(f" Sukces! Plik zapisany w: {file_to_save}")

Pobieram dane z NBP...
✅ Sukces! Plik zapisany w: /home/jovyan/work/data/bronze/raw_table_a.json


In [9]:
import requests
import json
from pathlib import Path

# 1. Definiujemy BEZWZGLĘDNĄ ścieżkę (taką, jaką widzi Jupyter)
# To jest ten sam folder, który masz na dysku C:\Projects\NBP_spark\data\bronze
bronze_path = Path("/home/jovyan/work/data/bronze")

# 2. Tworzymy folder, jeśli go nie ma (parents=True stworzy też folder 'data' jeśli trzeba)
bronze_path.mkdir(parents=True, exist_ok=True)

# 3. Pobieramy dane
print("Pobieram dane z NBP...")
raw_data_a = requests.get("http://api.nbp.pl/api/exchangerates/tables/b/?format=json").json()

# 4. Zapisujemy używając pełnej ścieżki
file_to_save = bronze_path / "raw_table_b.json"

with open(file_to_save, 'w', encoding='utf-8') as f:
    json.dump(raw_data_a, f, ensure_ascii=False, indent=4)

print(f" Sukces! Plik zapisany w: {file_to_save}")

Pobieram dane z NBP...
✅ Sukces! Plik zapisany w: /home/jovyan/work/data/bronze/raw_table_b.json


## Test if data is avaliable

In [5]:
import requests
from datetime import date

def check_nbp_data_availability():
    # 1. Get current system date
    today_date = date.today()
    today_str = str(today_date)
    
    print(f"System date: {today_str}")
    print("-" * 40)

    try:
        # 2. Check Exchange Rates (Table A)
        # Fetch the single most recent table metadata
        response_rates = requests.get("http://api.nbp.pl/api/exchangerates/tables/a/?format=json")
        
        if response_rates.status_code == 200:
            latest_api_date_rates = response_rates.json()[0]['effectiveDate']
            
            if latest_api_date_rates == today_str:
                print(f"Exchange Rates: Data for {today_str} is AVAILABLE in NBP API.")
            else:
                print(f"Exchange Rates: Data for {today_str} is NOT YET available. Latest: {latest_api_date_rates}")
        else:
            print(f"Exchange Rates: API request failed with status code {response_rates.status_code}")

        # 3. Check Gold Prices
        # Fetch the single most recent gold price data point
        response_gold = requests.get("http://api.nbp.pl/api/cenyzlota/last/1/?format=json")
        
        if response_gold.status_code == 200:
            latest_api_date_gold = response_gold.json()[0]['data']
            
            if latest_api_date_gold == today_str:
                print(f"Gold Prices: Data for {today_str} is AVAILABLE in NBP API.")
            else:
                print(f"Gold Prices: Data for {today_str} is NOT YET available. Latest: {latest_api_date_gold}")
        else:
            print(f"Gold Prices: API request failed with status code {response_gold.status_code}")

    except requests.exceptions.RequestException as e:
        print(f"Network error occurred: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Run the sensor
if __name__ == "__main__":
    check_nbp_data_availability()

System date: 2026-01-12
----------------------------------------
Exchange Rates: Data for 2026-01-12 is AVAILABLE in NBP API.
Gold Prices: Data for 2026-01-12 is AVAILABLE in NBP API.
